In [1]:
import torch
from torch import nn
import torch.nn.functional as F

# 1. Defining the CNN architecture

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=21, padding=10)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=15, padding=7)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=11, padding=5)
        
        self.batchnorm32 = nn.BatchNorm1d(num_features=32)
        self.batchnorm64 = nn.BatchNorm1d(num_features=64)
        self.batchnorm128 = nn.BatchNorm1d(num_features=128)
        self.batchnorm512 = nn.BatchNorm1d(num_features=512)
        
        self.fc1 = nn.Linear(4736, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1)
        
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.batchnorm32(x))
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = F.relu(self.batchnorm64(x))
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = F.relu(self.batchnorm128(x))
        x = self.maxpool(x)
        
        # Flatten input for fully connected layers
        x = x.view(x.shape[0], -1) 
        
        x = self.fc1(x)
        x = F.relu(self.batchnorm512(x))
        
        x = self.fc2(x)
        x = F.relu(self.batchnorm512(x))
        
        x = self.fc3(x)
        return x

# 2. Loading the Data

In [3]:
import h5py

traindata = h5py.File("../train/scsn_p_2000_2017_6sec_0.5r_pick_train_mix.hdf5")
testdata = h5py.File("../test/scsn_p_2000_2017_6sec_0.5r_pick_test_mix.hdf5")

In [4]:
from torch.utils.data import DataLoader

batch_size = 250

train_size = 10000
# val_size = train_size *
test_size = 2000

# Load test data
trainset = traindata["X"][:train_size]
trainlabels = traindata["pwave"][:train_size]

trainset = list(zip(trainset, trainlabels))

testset = testdata["X"][:test_size]
testlabels = testdata["pwave"][:test_size]

testset = list(zip(testset, testlabels))

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [6]:
model = CNN()

# 3. Add Multi-GPU Support to Model 
In order to run the model on multiple GPU's, we can use the nn.DataParellel method. This method requires that we move all tensors to the cuda:0 (the default gpu) before we can pass them through the network. 

In [7]:
def parallelize(model):
    device_ids = [i for i in range(torch.cuda.device_count())]
    model = torch.nn.DataParallel(model, device_ids=device_ids)
    return model

In [8]:
model = parallelize(model)
model

DataParallel(
  (module): CNN(
    (conv1): Conv1d(1, 32, kernel_size=(21,), stride=(1,), padding=(10,))
    (conv2): Conv1d(32, 64, kernel_size=(15,), stride=(1,), padding=(7,))
    (conv3): Conv1d(64, 128, kernel_size=(11,), stride=(1,), padding=(5,))
    (batchnorm32): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchnorm64): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchnorm128): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchnorm512): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=4736, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=1, bias=True)
    (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

# 4. Define Loss Function and Optimizer
Here we define the loss function and optimizer. For the loss function (criterion), we use the binary cross entropy with logits loss (BCEWithLogitsLoss)

In [9]:
from torch import optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. Training and Validation

In [10]:
epochs = 30

training_losses = []
validation_losses = []

for epoch in range(epochs):
    model.train()
    train_loss = 0
    
    for batch, labels in trainloader:
        # ============================================
        #            TRAINING
        # ============================================
        batch, labels = batch.to(device), labels.to(device)
        # Clear gradients in optimizer
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(batch.unsqueeze(1))
        # Calculate loss
        loss = criterion(output.squeeze(), labels.type(torch.cuda.FloatTensor))
        train_loss += loss.item()
        # Backpropagation
        loss.backward()
        # Update weights
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            validation_loss = 0

            for batch, labels in testloader:
                # ============================================
                #            VALIDATION
                # ============================================
                batch, labels = batch.to(device), labels.to(device)
                # Forward pass
                ouput = model.forward(batch.unsqueeze(1))
                # Calculate loss
                loss = criterion(output.squeeze(), labels.type(torch.cuda.FloatTensor))
                validation_loss += loss.item()
                
    # Print epoch summary
    t_loss_avg = train_loss / len(trainloader)
    v_loss_avg = validation_loss / len(testloader)
    
    training_losses.append(t_loss_avg)
    validation_losses.append(v_loss_avg)
    
    print('Epoch [{:5d}/{:5d}] | train loss: {:6.4f} | validation loss: {:6.4f}'.format(
            epoch+1, epochs, t_loss_avg, v_loss_avg))
        

Epoch [    1/   30] | train loss: 0.1216 | validation loss: 2.2347
Epoch [    2/   30] | train loss: 0.0525 | validation loss: 2.6065
Epoch [    3/   30] | train loss: 0.0416 | validation loss: 2.7682
Epoch [    4/   30] | train loss: 0.0309 | validation loss: 3.1524
Epoch [    5/   30] | train loss: 0.0250 | validation loss: 3.2717
Epoch [    6/   30] | train loss: 0.0175 | validation loss: 3.5634
Epoch [    7/   30] | train loss: 0.0124 | validation loss: 3.6730
Epoch [    8/   30] | train loss: 0.0140 | validation loss: 4.0407
Epoch [    9/   30] | train loss: 0.0110 | validation loss: 3.8949
Epoch [   10/   30] | train loss: 0.0075 | validation loss: 4.0702
Epoch [   11/   30] | train loss: 0.0041 | validation loss: 4.1698
Epoch [   12/   30] | train loss: 0.0070 | validation loss: 4.2465
Epoch [   13/   30] | train loss: 0.0073 | validation loss: 4.6131
Epoch [   14/   30] | train loss: 0.0068 | validation loss: 4.5385
Epoch [   15/   30] | train loss: 0.0015 | validation loss: 4.

# References

1. https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2017JB015251
2. http://scedc.caltech.edu/research-tools/deeplearning.html#picking_polarity